## Knowledge-based Recommender System

In [4]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances

import matplotlib as plt

import re

In [5]:
fl = pd.read_csv('../data/consumer_airfare_report_US.csv')

/var/folders/7q/jsl03b552s5fnhvvhgv3sz100000gn/T/ipykernel_28938/3634177382.py:1: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  fl = pd.read_csv('../data/consumer_airfare_report_US.csv')


In [6]:
fl.head()

,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,...,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,Geocoded_City1,Geocoded_City2,tbl1apk
0,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,14112,ABE,...,81.43,G4,1.0000,81.43,G4,1.0000,81.43,NaN,NaN,202131013514112ABEPIE
1,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,15304,ABE,...,208.93,DL,0.4659,219.98,UA,0.1193,154.11,NaN,NaN,202131013515304ABETPA
2,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11259,ABQ,...,184.56,WN,0.9968,184.44,WN,0.9968,184.44,NaN,NaN,202131014011259ABQDAL
3,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11298,ABQ,...,182.64,AA,0.9774,183.09,AA,0.9774,183.09,NaN,NaN,202131014011298ABQDFW
4,Table1a,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",10140,14107,ABQ,...,177.11,WN,0.6061,184.49,AA,0.3939,165.77,NaN,NaN,202131014014107ABQPHX


In [7]:
fl.shape

(244050, 23)

In [8]:
fl.columns

Index(['tbl', 'Year', 'quarter', 'citymarketid_1', 'citymarketid_2', 'city1',
       'city2', 'airportid_1', 'airportid_2', 'airport_1', 'airport_2',
       'nsmiles', 'passengers', 'fare', 'carrier_lg', 'large_ms', 'fare_lg',
       'carrier_low', 'lf_ms', 'fare_low', 'Geocoded_City1', 'Geocoded_City2',
       'tbl1apk'],
      dtype='object')

In [9]:
fl['Year'].unique()

array([2021, 2022, 2010, 1998, 2009, 1993, 2007, 2003, 2005, 2014, 2001,
       2002, 2011, 1999, 2012, 2013, 1997, 2006, 1994, 1996, 2004, 2000,
       2008, 2019, 2015, 2017, 2016, 2023, 2018, 2020])

In [10]:
fl.drop(columns= ['tbl', 'Geocoded_City1', 'Geocoded_City2',
       'tbl1apk', 'carrier_lg', 'large_ms', 'fare_lg',
       'carrier_low', 'airportid_1', 'airportid_2', 'airport_1', 'airport_2', 'nsmiles', 'lf_ms'], inplace = True)

In [11]:
fl.head()

,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,passengers,fare,fare_low
0,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",180,81.43,81.43
1,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",19,208.93,154.11
2,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",204,184.56,184.44
3,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",264,182.64,183.09
4,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",398,177.11,165.77


In [12]:
fl.shape

(244050, 9)

In [13]:
fl.isnull().sum()

Year                 0
quarter              0
citymarketid_1       0
citymarketid_2       0
city1                0
city2                0
passengers           0
fare                 0
fare_low          1606
dtype: int64

In [14]:
def camel_to_snake(col):
    return re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower()

In [15]:
fl.columns = [camel_to_snake(col) for col in fl.columns]

citymarketid_1 = City market ID is an identification number assigned by US DOT to identify a city market. Use this field to consolidate airports serving the same city market
citymarketid_2 = 
nsmiles = Non-Stop market miles
passenger = passenger per day
fare = overall average fare
fare_low = Average fare for the lowest carrier

In [17]:
#dropping nulls in fare_low
fl.dropna()

,year,quarter,citymarketid_1,citymarketid_2,city1,city2,passengers,fare,fare_low
0,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",180,81.43,81.43
1,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",19,208.93,154.11
2,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",204,184.56,184.44
3,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",264,182.64,183.09
4,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",398,177.11,165.77
...,...,...,...,...,...,...,...,...,...
244045,2023,4,35412,31703,"Knoxville, TN","New York City, NY (Metropolitan Area)",251,285.28,243.20
244046,2023,4,35412,32467,"Knoxville, TN","Miami, FL (Metropolitan Area)",261,146.87,116.00
244047,2023,4,35412,32467,"Knoxville, TN","Miami, FL (Metropolitan Area)",86,281.16,259.68
244048,2023,4,35412,33195,"Knoxville, TN","Tampa, FL (Metropolitan Area)",237,92.75,92.75


In [18]:
fl['year'].value_counts()

year
1993    9739
1996    9081
1997    8949
1999    8757
1998    8708
2001    8648
2002    8589
2000    8541
2003    8488
2004    8466
2005    8365
2013    8301
2012    8238
2017    8232
2014    8229
2016    8227
2006    8200
2018    8195
2015    8150
2019    8148
2011    8144
2010    8113
2009    8109
2008    8107
2007    7997
2022    7809
2023    7788
2021    7758
2020    7520
1994    2454
Name: count, dtype: int64

In [19]:
fl.drop(columns= ['quarter', 'citymarketid_1', 'citymarketid_2', 'passengers', 'fare_low'], inplace = True)

In [20]:
#only interested in years 2020-2023 - other fares are more than likely outdated
years_to_keep = [2020, 2021, 2022, 2023]
fl_filtered = fl[fl['year'].isin(years_to_keep)]

In [21]:
fl_filtered.shape

(30875, 4)

In [22]:
fl_filtered['year'].value_counts()

year
2022    7809
2023    7788
2021    7758
2020    7520
Name: count, dtype: int64

In [23]:
fl_filtered.head()

,year,city1,city2,fare
0,2021,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",81.43
1,2021,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",208.93
2,2021,"Albuquerque, NM","Dallas/Fort Worth, TX",184.56
3,2021,"Albuquerque, NM","Dallas/Fort Worth, TX",182.64
4,2021,"Albuquerque, NM","Phoenix, AZ",177.11


In [24]:
fl_filtered['city1'].value_counts

<bound method IndexOpsMixin.value_counts of 0         Allentown/Bethlehem/Easton, PA
1         Allentown/Bethlehem/Easton, PA
2                        Albuquerque, NM
3                        Albuquerque, NM
4                        Albuquerque, NM
                       ...              
244045                     Knoxville, TN
244046                     Knoxville, TN
244047                     Knoxville, TN
244048                     Knoxville, TN
244049                     Knoxville, TN
Name: city1, Length: 30875, dtype: object>

In [25]:
#separate city from state
fl_filtered[['city1', 'state1']] = fl.loc[:, 'city1'].str.split(', ',  n=1, expand=True)
fl_filtered[['city2', 'state2']] = fl.loc[:, 'city2'].str.split(', ', n=1, expand=True)

/var/folders/7q/jsl03b552s5fnhvvhgv3sz100000gn/T/ipykernel_28938/434849404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_filtered[['city1', 'state1']] = fl.loc[:, 'city1'].str.split(', ',  n=1, expand=True)
/var/folders/7q/jsl03b552s5fnhvvhgv3sz100000gn/T/ipykernel_28938/434849404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_filtered[['city1', 'state1']] = fl.loc[:, 'city1'].str.split(', ',  n=1, expand=True)
/var/folders/7q/jsl03b552s5fnhvvhgv3sz100000gn/T/ipykernel_28938/434849404.py:3:

In [26]:
fl_filtered.isnull().sum()

year      0
city1     0
city2     0
fare      0
state1    0
state2    0
dtype: int64

In [27]:
fl_filtered.shape

(30875, 6)

In [28]:
fl_filtered.to_csv('../data/flight_data_clean.csv', index=False) 

In [29]:
fl_filtered['city1'].unique()

array(['Allentown/Bethlehem/Easton', 'Albuquerque', 'Nantucket',
       'Colorado Springs', 'Dallas/Fort Worth', 'Pittsburgh',
       'Huntsville', 'Albany', 'Amarillo', 'Denver', 'Atlanta', 'Austin',
       'Asheville', 'Tucson', 'Phoenix', 'Hartford', 'Seattle',
       'Birmingham', 'El Paso', 'Cleveland', 'Nashville', 'Boise',
       'Boston', 'Burlington', 'Buffalo', 'Bozeman', 'Washington',
       'Chicago', 'Charleston', 'Cedar Rapids/Iowa City', 'Charlotte',
       'Columbus', 'St. Louis', 'Myrtle Beach', 'Jacksonville', 'Detroit',
       'Des Moines', 'Houston', 'Orlando', 'Panama City', 'Valparaiso',
       'Eugene', 'Key West', 'Fargo', 'Kalispell', 'Minneapolis/St. Paul',
       'New York City', 'Fort Myers', 'Greenville/Spartanburg',
       'Grand Rapids', 'Greensboro/High Point', 'Las Vegas',
       'Indianapolis', 'Jackson', 'Jackson/Vicksburg', 'San Francisco',
       'Miami', 'Los Angeles', 'Little Rock', 'Louisville', 'Cincinnati',
       'Sacramento', 'Tampa', 'Kansas

In [30]:
fl_filtered['fare'].value_counts()

fare
200.76    11
206.00     9
183.89     9
238.00     8
179.50     8
          ..
350.81     1
209.22     1
361.77     1
389.03     1
342.13     1
Name: count, Length: 18128, dtype: int64

In [31]:
#keeping only city2 that matches cities of airbnb
cities_to_keep = ['Austin', 'Boston', 'Chicago', 'Columbus', 'Denver', 'Ft Lauderdale', 
    'honolulu', 'Jersey City', 'Las Vegas', 'Los Angeles', 'Nashville', 
    'New Orleans', 'New York City', 'Portland', 'Rhode Island', 
    'San Diego', 'San Francisco', 'Seattle', 'Minneapolis', 'Washington']              

In [32]:
filtered_df = fl_filtered[fl_filtered['city2'].isin(cities_to_keep)]

In [33]:
filtered_df.shape

(16099, 6)

In [34]:
filtered_df.to_csv('../data/flight_data_clean.csv', index=False)

## Pivot Table created but ultimately not used.  

In [36]:
fare_mx = pd.pivot_table(
    fl_filtered,
    values = 'fare',
    index = 'city2',
    columns = 'city1'
)

In [37]:
fare_mx.head()

city1,Albany,Albuquerque,Allentown/Bethlehem/Easton,Amarillo,Appleton,Asheville,Ashland,Aspen,Atlanta,Atlantic City,...,South Bend,Spokane,Springfield,St. Louis,Syracuse,Tampa,Tucson,Tulsa,Valparaiso,Washington
city2,,,,,,,,,,,,,,,,,,,,,
Boston,NaN,345.422222,NaN,NaN,NaN,218.314444,NaN,NaN,216.9025,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bozeman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Buffalo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Charleston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Charlotte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
fare_mx.shape

(109, 124)

In [39]:
fl_filtered.columns

Index(['year', 'city1', 'city2', 'fare', 'state1', 'state2'], dtype='object')

Sparse Matrix

In [41]:
fl_filtered['fare'].value_counts()

fare
200.76    11
206.00     9
183.89     9
238.00     8
179.50     8
          ..
350.81     1
209.22     1
361.77     1
389.03     1
342.13     1
Name: count, Length: 18128, dtype: int64

In [42]:
fare_sparse = sparse.csr_matrix(fare_mx.fillna(0))

In [43]:
print(fare_sparse)

  (0, 1)	345.4222222222222
  (0, 5)	218.31444444444443
  (0, 8)	216.9025
  (0, 10)	242.35833333333335
  (1, 19)	316.0680952380952
  (2, 19)	199.4039393939394
  (3, 19)	193.22
  (4, 19)	213.0822916666667
  (5, 0)	202.07227272727275
  (5, 1)	243.272
  (5, 5)	153.8075
  (5, 7)	240.03
  (5, 8)	157.718125
  (5, 10)	187.0178125
  (5, 16)	193.92633333333333
  (5, 18)	261.38
  (5, 19)	187.52166666666668
  (5, 20)	265.59727272727275
  (5, 21)	169.85142857142858
  (5, 22)	340.8925
  (5, 24)	185.0421875
  (5, 25)	175.10875
  (6, 19)	213.04833333333332
  (6, 27)	159.633125
  (7, 8)	194.095
  :	:
  (104, 106)	322.62
  (104, 107)	256.27208333333334
  (104, 108)	306.21979166666665
  (104, 109)	309.34027777777777
  (104, 110)	181.81333333333336
  (104, 111)	191.21062500000002
  (104, 112)	292.169375
  (104, 115)	211.38
  (104, 117)	197.51104166666667
  (104, 118)	182.99366666666668
  (104, 119)	186.78080000000003
  (104, 120)	315.6504166666667
  (104, 121)	213.24333333333334
  (104, 122)	224.062222222

In [44]:
import sys

In [45]:
sys.getsizeof(fare_mx)

115495

In [46]:
sys.getsizeof(fare_sparse)

56

Cosine Similarity

In [48]:
# distance of 1 is a similarity of 0.
dists = pairwise_distances(fare_sparse, metric='cosine')

In [49]:
print(dists.shape)
dists[:10, :10]

(109, 109)


array([[0.        , 1.        , 1.        , 1.        , 1.        ,
        0.53631309, 1.        , 0.61254082, 1.        , 0.60547614],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.76965276, 0.199724  , 0.5715769 , 1.        , 0.53164062],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.76965276, 0.199724  , 0.5715769 , 1.        , 0.53164062],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.76965276, 0.199724  , 0.5715769 , 1.        , 0.53164062],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.76965276, 0.199724  , 0.5715769 , 1.        , 0.53164062],
       [0.53631309, 0.76965276, 0.76965276, 0.76965276, 0.76965276,
        0.        , 0.81565863, 0.54188271, 1.        , 0.5952323 ],
       [1.        , 0.199724  , 0.199724  , 0.199724  , 0.199724  ,
        0.81565863, 0.        , 0.44989571, 0.40036818, 0.40651601],
       [0.61254082, 0.5715769 , 0.5715769

Distances DataFrame

In [51]:
fl_rec = pd.DataFrame(dists, columns=fare_mx.index, index = fare_mx.index)

In [52]:
fl_rec.iloc[:10, :10]

city2,Boston,Bozeman,Buffalo,Charleston,Charlotte,Chicago,Cincinnati,Cleveland,Colorado Springs,Columbus
city2,,,,,,,,,,
Boston,0.000000,1.000000,1.000000,1.000000,1.000000,0.536313,1.000000,0.612541,1.000000,0.605476
Bozeman,1.000000,0.000000,0.000000,0.000000,0.000000,0.769653,0.199724,0.571577,1.000000,0.531641
Buffalo,1.000000,0.000000,0.000000,0.000000,0.000000,0.769653,0.199724,0.571577,1.000000,0.531641
Charleston,1.000000,0.000000,0.000000,0.000000,0.000000,0.769653,0.199724,0.571577,1.000000,0.531641
Charlotte,1.000000,0.000000,0.000000,0.000000,0.000000,0.769653,0.199724,0.571577,1.000000,0.531641
Chicago,0.536313,0.769653,0.769653,0.769653,0.769653,0.000000,0.815659,0.541883,1.000000,0.595232
Cincinnati,1.000000,0.199724,0.199724,0.199724,0.199724,0.815659,0.000000,0.449896,0.400368,0.406516
Cleveland,0.612541,0.571577,0.571577,0.571577,0.571577,0.541883,0.449896,0.000000,0.654375,0.063229
Colorado Springs,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.400368,0.654375,0.000000,0.635331


In [53]:
starting_city = fare_mx.index

starting_city[starting_city.str.lower().str.contains('chicago')]

Index(['Chicago'], dtype='object', name='city2')

In [54]:
fl_rec['Chicago'].sort_values().iloc[1:].head(10)

city2
Dallas/Fort Worth    0.317262
Los Angeles          0.530974
New York City        0.532197
Boston               0.536313
Cleveland            0.541883
Columbus             0.595232
Houston              0.601137
Miami                0.630277
Washington           0.642268
Phoenix              0.678234
Name: Chicago, dtype: float64

In [55]:
starting_city[starting_city.str.lower().str.contains('las vegas')]

Index(['Las Vegas'], dtype='object', name='city2')

In [56]:
fl_rec['Las Vegas'].sort_values().iloc[1:5]

city2
Denver                  0.068280
Fort Myers              0.082577
Philadelphia            0.157329
Minneapolis/St. Paul    0.161650
Name: Las Vegas, dtype: float64